In [32]:
import re
import pandas as pd
import phrasemachine
import nltk
from rake_nltk import Rake
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import ngrams, FreqDist

In [3]:
reviews = pd.read_csv('/Users/dylanhayashi/Desktop/Northwestern/NU_MSDS/453 - Natural Language Processing/453.1 - Natural Language Processing and Artificial Intelligence/DHB_MovieReview.csv')

In [33]:
# Only run this once, they will be downloaded.
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dylanhayashi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dylanhayashi/nltk_data...


True

In [6]:
review_one = reviews.loc[0,'MovieReview']
review_two = reviews.loc[1,'MovieReview']
review_three = reviews.loc[2,'MovieReview']
review_four = reviews.loc[3,'MovieReview']
review_five = reviews.loc[4,'MovieReview']
review_six = reviews.loc[5,'MovieReview']
review_seven = reviews.loc[6,'MovieReview']
review_eight = reviews.loc[7,'MovieReview']
review_nine = reviews.loc[8,'MovieReview']
review_ten = reviews.loc[0,'MovieReview']

In [25]:
term_list = [
    'Paris','Liam Neeson','Liam','Neeson','CIA','Central Intelligence Agency','Albania','Europe','U2','Hollywood','Bryan',
    'kidnap','action','fight','scenen','slavery','abduct','kill','torture','shoot','cliche','flight','vacation','victim',
    'women','men','police','drug','trauma','character','airport','thrill','moral','human','traffick','trailer','review',
    'rating','dialogue','production','direct','cinematography','PG-13','review','film','movie','spy','act','suspense',
    'good','bad','best','worst','offensive','realistic','believable','terrible','great','again','watch','see','crime',
    'organized crime'
]

In [28]:
term_frequencies = pd.DataFrame(columns=['Term','doc1_count','doc2_count','doc3_count','doc4_count','doc5_count','doc6_count','doc7_count','doc8_count','doc9_count','doc10_count'])

for term in term_list:

    frequencies = pd.DataFrame(
        [{
            'Term':term,
            'doc1_count':len(re.findall(term,review_one)),
            'doc2_count':len(re.findall(term,review_two)),
            'doc3_count':len(re.findall(term,review_three)),
            'doc4_count':len(re.findall(term,review_four)),
            'doc5_count':len(re.findall(term,review_five)),
            'doc6_count':len(re.findall(term,review_six)),
            'doc7_count':len(re.findall(term,review_seven)),
            'doc8_count':len(re.findall(term,review_eight)),
            'doc9_count':len(re.findall(term,review_nine)),
            'doc10_count':len(re.findall(term,review_ten))
        }]
    )

    term_frequencies = pd.concat([term_frequencies,frequencies])

term_frequencies

,Term,doc1_count,doc2_count,doc3_count,doc4_count,doc5_count,doc6_count,doc7_count,doc8_count,doc9_count,doc10_count
0,Paris,6,1,11,0,0,0,1,2,1,6
0,Liam Neeson,0,0,0,0,1,4,3,2,3,0
0,Liam,0,0,12,0,1,4,4,4,3,0
0,Neeson,0,5,0,0,3,7,6,2,4,0
0,CIA,1,1,5,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
0,terrible,1,0,0,0,0,0,0,0,0,1
0,great,0,0,0,1,0,5,0,3,0,0
0,again,1,0,0,0,0,0,1,0,0,1
0,watch,2,1,0,0,2,0,3,0,0,2


In [34]:
# Create a list of stop words from nltk
stop_words = set(stopwords.words("english"))

# Pre-process dataset to remove punctuation
def remove_punctuation(in_text):
    # Remove punctuation
    text = re.sub('[^a-zA-Z]', ' ', str(in_text))
    return text

# Pre-process dataset to lower case it
def lower_case(in_text):
    # Convert to lowercase
    text = in_text.lower()    
    return text

# Pre-process dataset to remove tags
def remove_tags(in_text):    
    # Remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",in_text)
    return text

# Pre-process dataset to remove special characters and digits
def remove_special_chars_and_digits(in_text):
    # Remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",in_text)
    return text

# Pre-process dataset to appy Stemming
def apply_stemming(in_text):
    stemmer=PorterStemmer()
    word_list = nltk.word_tokenize(in_text)
    output = ' '.join([stemmer.stem(w) for w in word_list])
    return output

# Pre-process dataset to apply Lemmatization
def apply_lemmatization(in_text):
    # Lemmatization
    lem = WordNetLemmatizer()
    word_list = nltk.word_tokenize(in_text)
    output = ' '.join([lem.lemmatize(w) for w in word_list])
    return output

# Remove stop words
def remove_stop_words(in_text):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(in_text)  
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 

    return filtered_sentence

# Run Phase Machine
def run_phrase_machine(in_text):
    phrases=phrasemachine.get_phrases(in_text)
    return phrases

#Run Rake Keyword Extractor
def run_rake(in_text):
    r = Rake()
    r.extract_keywords_from_text(in_text)
    rake_phrases= r.get_ranked_phrases()
    return rake_phrases

# Run NLTK Tokenizer
def run_nltk_tokenizer(in_text):
    tokens=nltk.word_tokenize(in_text)
    return tokens

# Run NLTK Sentence Tokenizer
def run_nltk_sent_tokenizer(in_corpus):
    sents = nltk.sent_tokenize(in_corpus)
    return sents

#Run word-ngram Tokenizer
def run_nltk_tokenizer_word_ngrams(in_text, ngram_size):
    n_grams = ngrams(nltk.word_tokenize(in_text), ngram_size)
    return [ ' '.join(grams) for grams in n_grams]

#Get Frequ Dist 
def get_freq_dist(terms):
    all_counts = dict()
    all_counts = FreqDist(terms)
    return all_counts

In [35]:
sentences_one = run_nltk_sent_tokenizer(review_one)
sentences_two = run_nltk_sent_tokenizer(review_two)
sentences_three = run_nltk_sent_tokenizer(review_three)
sentences_four = run_nltk_sent_tokenizer(review_four)
sentences_five = run_nltk_sent_tokenizer(review_five)
sentences_six = run_nltk_sent_tokenizer(review_six)
sentences_seven = run_nltk_sent_tokenizer(review_seven)
sentences_eight = run_nltk_sent_tokenizer(review_eight)
sentences_nine = run_nltk_sent_tokenizer(review_nine)
sentences_ten = run_nltk_sent_tokenizer(review_ten)

In [37]:
#Explore different extractors and difference preprocessing techniques
for sentence in sentences_one:
    print(sentence)
    print("===================NLTK Tokenizer===================")
    print(run_nltk_tokenizer(sentence))
    print("===================NLTK Word NGRAM Tokenizer 2 words===================")
    print(run_nltk_tokenizer_word_ngrams(sentence,2))
    print("===================NLTK Word NGRAM Tokenizer 3 words===================")
    print(run_nltk_tokenizer_word_ngrams(sentence,3))
    print("===================Phrase Machine===================")
    phrases=run_phrase_machine(sentence)
    for term in phrases["counts"].keys():
        print(term)
    print("===================Rake===================")
    print(run_rake(sentence))
    print("===================NLTK Tokenizer===================")
    print(run_nltk_tokenizer((sentence)))
    print("===================NLTK Tokenizer LOWER CASE===================")
    print(run_nltk_tokenizer(lower_case(sentence)))
    print("===================NLTK Tokenizer REMOVE STOP WORDS===================")
    print(remove_stop_words(sentence))   
    print("===================NLTK Tokenizer REMOVED PUNCTUATION===================")
    print(run_nltk_tokenizer(remove_punctuation(sentence)))
    print("===================NLTK Tokenizer REMOVED TAGS===================")
    print(run_nltk_tokenizer(remove_tags(sentence)))
    print("===================NLTK Tokenizer REMOVED CHARS AND DIGITS===================")
    print(run_nltk_tokenizer(remove_special_chars_and_digits(sentence)))
    print("===================NLTK Tokenizer STEMMING APPLIED===================")
    print(run_nltk_tokenizer(apply_stemming(sentence)))
    print("===================NLTK Tokenizer LEMMATIZATION APPLIED===================")
    print(run_nltk_tokenizer(apply_lemmatization(sentence)))
    #break

To save you the time in either watching this film or reading my critique of it, here is a brief summary: This film is the worst film I have ever seen.
===================NLTK Tokenizer===================
['To', 'save', 'you', 'the', 'time', 'in', 'either', 'watching', 'this', 'film', 'or', 'reading', 'my', 'critique', 'of', 'it', ',', 'here', 'is', 'a', 'brief', 'summary', ':', 'This', 'film', 'is', 'the', 'worst', 'film', 'I', 'have', 'ever', 'seen', '.']
===================NLTK Word NGRAM Tokenizer 2 words===================
['To save', 'save you', 'you the', 'the time', 'time in', 'in either', 'either watching', 'watching this', 'this film', 'film or', 'or reading', 'reading my', 'my critique', 'critique of', 'of it', 'it ,', ', here', 'here is', 'is a', 'a brief', 'brief summary', 'summary :', ': This', 'This film', 'film is', 'is the', 'the worst', 'worst film', 'film I', 'I have', 'have ever', 'ever seen', 'seen .']
===================NLTK Word NGRAM Tokenizer 3 words============

In [45]:
#Explore different extractors and difference preprocessing techniques
all_terms=[]
for sentence in sentences_nine:
    print(sentence)
    #pick your favorite term extractor
    all_terms = all_terms +run_rake(sentence)
#get the frequency distribution across the terms
fd=get_freq_dist(all_terms)
fd


The latest rescue-slash-revenge film to hit our screens, Taken, tries to walk the fine line between the two classic character archetypes - family man and ultimate badass.
When the estranged seventeen year old daughter of a former spy (Liam Neeson) is kidnapped while holidaying overseas, her father must call upon his unique skill set in order to rescue her before she becomes lost to the Eastern-European slave trade.
Liam Neeson kicking ass, and the potential of Maggie Grace showing a little - Taken had my full and undivided attention.
As with the majority of films that limp onto the silver screen these days, everything seems to be a remake of a remake, or at the very least, a rehash of a tired idea.
What separates a stock-standard retelling, from something that will stay with you longer than a half-masticated corn kernel stuck in your teeth, will almost certainly come down to the execution.
Take a well-worn concept, freshen it up a little, but ultimately pull it off with dedication and 

FreqDist({'taken': 6, 'well': 5, 'bryan': 4, 'get': 4, 'daughter': 4, 'little': 3, 'point': 3, 'liam neeson': 2, 'former spy': 2, 'kidnapped': 2, ...})